# Quick Demo to showcase how to use prior knowledge in rCausalMGM

In [1]:
library(rCausalMGM)

data(train_n10000_p10)
data(dag_n10000_p10)

print(head(train_n10000_p10))

          X1          X2         X3         X4         X5 Y1 Y2 Y3 Y4 Y5
1 -0.1032843  1.51656848  1.6732801 -0.4708008  0.9999730  A  B  A  B  A
2  0.3748836 -0.38811637 -0.6579513 -0.7048476  1.3844457  A  B  A  B  A
3 -1.3737184  0.03107079  0.7662326  0.5035131 -0.5010449  A  A  B  B  A
4  2.3889562 -1.36889096 -1.1946817  2.0358353  0.9168064  A  A  B  C  B
5 -1.3785314 -0.13704609 -2.4455097  1.8642418  0.4437788  C  A  C  B  A
6  1.3540231 -0.65017245  0.2355710 -0.5957924  0.6637254  A  A  C  C  C


## Lets shrink the data so the algos can run quickly

In [2]:
dat <- train_n10000_p10[1:10, ]

print(dat)

            X1          X2         X3         X4         X5 Y1 Y2 Y3 Y4 Y5
1  -0.10328431  1.51656848  1.6732801 -0.4708008  0.9999730  A  B  A  B  A
2   0.37488364 -0.38811637 -0.6579513 -0.7048476  1.3844457  A  B  A  B  A
3  -1.37371842  0.03107079  0.7662326  0.5035131 -0.5010449  A  A  B  B  A
4   2.38895617 -1.36889096 -1.1946817  2.0358353  0.9168064  A  A  B  C  B
5  -1.37853143 -0.13704609 -2.4455097  1.8642418  0.4437788  C  A  C  B  A
6   1.35402307 -0.65017245  0.2355710 -0.5957924  0.6637254  A  A  C  C  C
7  -2.15890719 -3.59002886 -1.2250584  2.5134897 -2.0204028  A  C  A  C  A
8   2.01495897 -5.21212079  0.6088620  0.4483583 -2.6596373  B  C  A  C  C
9  -0.42249171 -1.70941725  1.5490180 -0.4158757 -0.6989711  A  A  C  A  A
10 -0.09060777  1.88340600  1.5285464 -1.5011507  0.7501029  C  B  B  A  A


In [3]:
g <- rCausalMGM::pcStable(data = dat, verbose = T)

message("Here is the edge list...")

g$edges

Starting PC-Stable algorithm...
  Starting FAS Stable...
    Searching at depth 0...
    Searching at depth 1...
  FAS Stable Elapsed Time =  0.008 s
    Filling Triple Map...
  Orienting colliders...
  Orienting implied edges...
  PC-Stable Elapsed Time =  0.01 s


Here is the edge list...



[1] "X1 --- Y5" "X3 --- X4"

## The first thing we should examine is the knowledge 'tier' system
- It should be the case that all of the nodes on the same tier should not affect the learned edge list

In [236]:
g <- rCausalMGM::pcStable(
    data = dat,
    verbose = T,
    
    # Try knowledge
    knowledge  = list(
        
        "tiers" = list(
            c('X5', 'X4', 'X2', 'X1', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'X3')
        )
        
    )
)

message("Here is the edge list...")

g$edges 

Starting PC-Stable algorithm...
  Starting FAS Stable...
    Searching at depth 0...
    Searching at depth 1...
  FAS Stable Elapsed Time =  0.0091 s
    Filling Triple Map...
  Orienting colliders...
  Orienting implied edges...
  PC-Stable Elapsed Time =  0.012 s


Here is the edge list...



[1] "X1 --- Y5" "X3 --- X4"

Nice!

## Let's look at how to do the tiered constraints!

In [222]:
g <- rCausalMGM::pcStable(
    data = dat,
    verbose = T,
    
    # Try knowledge
    knowledge  = list(
        
        "tiers" = list(
            c('X5', 'X4', 'X3', 'X2', 'X1'),
            c('Y1', 'Y2', 'Y3', 'Y4', 'Y5')
        )
        
    )
)

message("Here is the edge list...")

g$edges 

Starting PC-Stable algorithm...
  Starting FAS Stable...
    Searching at depth 0...
    Searching at depth 1...
  FAS Stable Elapsed Time =  0.0091 s
    Filling Triple Map...
  Orienting colliders...
  Orienting implied edges...
  PC-Stable Elapsed Time =  0.012 s


Here is the edge list...



[1] "X1 --> Y5" "X2 --- X5" "X3 --- X4"

## We can include a flag per tier to control whether the nodes can connect within!

In [4]:
g <- rCausalMGM::pcStable(
    data = dat,
    verbose = T,
    knowledge = list(

        "tiers" = list(
            c('X5', 'X4', 'X3', 'X2', 'X1'),
            c('Y1', 'Y2', 'Y3', 'Y4', 'Y5')
        ),

        "forbiddenWithinTier" = c(TRUE, TRUE)
    )
)
 
message("Here is the edge list...")

g$edges 

Starting PC-Stable algorithm...
  Starting FAS Stable...
    Searching at depth 0...
    Searching at depth 1...
  FAS Stable Elapsed Time =  0.0097 s
    Filling Triple Map...
  Orienting colliders...
  Orienting implied edges...
  PC-Stable Elapsed Time =  0.013 s


Here is the edge list...



[1] "X1 --> Y5" "X2 --> Y1"

'X2 --- X5' got deleted!

## Lets forbid two nodes from being connected! Passed as you would think!

In [226]:
g <- rCausalMGM::pcStable(
    data = dat,
    verbose = T,
    knowledge = list(

        "tiers" = list(
            c('X1', 'X2', 'X3', 'X4', 'X5'),
            c('Y1', 'Y2', 'Y3', 'Y4', 'Y5')
        ),

        "forbiddenWithinTier" = c(TRUE, TRUE),
        
        "forbidden" = list(
            c("X2", "Y1")
        )
        
    )
)
 
message("Here is the edge list...")

g$edges

Starting PC-Stable algorithm...
  Starting FAS Stable...
    Searching at depth 0...
    Searching at depth 1...
  FAS Stable Elapsed Time =  0.0089 s
    Filling Triple Map...
  Orienting colliders...
  Orienting implied edges...
  PC-Stable Elapsed Time =  0.011 s


Here is the edge list...



[1] "X1 --> Y5"

Look at that! It removed 'X2 --> Y1'

## Finally, let us *require* and edge between two nodes

In [229]:
g <- rCausalMGM::pcStable(
    data = dat,
    verbose = T,
    knowledge = list(

        "tiers" = list(
            c('X1', 'X2', 'X3', 'X4', 'X5'),
            c('Y1', 'Y2', 'Y3', 'Y4', 'Y5')
        ),

        "forbiddenWithinTier" = c(TRUE, TRUE),
        
        "forbidden" = list(
            c("X2", "Y1")
        ),
        
         "required" = list(
            c("X3", "Y5")
        )
        
    )
)
 
message("Here is the edge list...")

g$edges

Starting PC-Stable algorithm...
  Starting FAS Stable...
    Searching at depth 0...
    Searching at depth 1...
  FAS Stable Elapsed Time =  0.0086 s
    Filling Triple Map...
  Orienting colliders...
  Orienting implied edges...
  PC-Stable Elapsed Time =  0.011 s


Here is the edge list...



[1] "X1 --> Y5" "X3 --> Y5"

-----